In [ ]:
from Go2Py.robot.interface.dds import GO2Real
import numpy as np
robot = GO2Real(mode='lowlevel')

In [ ]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import KeyboardRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.control.walk_these_ways import *
import numpy as np

In [ ]:
class walkTheseWaysController:
    def __init__(self, robot, remote, checkpoint):
        self.remote = remote
        self.robot = robot
        self.cfg = loadParameters(checkpoint)
        self.policy = Policy(checkpoint)
        self.command_profile = CommandInterface()
        self.agent = WalkTheseWaysAgent(self.cfg, self.command_profile, self.robot)
        self.agent = HistoryWrapper(self.agent)
        self.init()

    def init(self):
        self.obs = self.agent.reset()
        self.policy_info = {}
        self.command_profile.yaw_vel_cmd = 0.0
        self.command_profile.x_vel_cmd = 0.0
        self.command_profile.y_vel_cmd = 0.0
        self.command_profile.stance_width_cmd=0.25
        self.command_profile.footswing_height_cmd=0.08
        self.command_profile.step_frequency_cmd = 3.0
        self.command_profile.bodyHeight = 0.00

    def update(self, robot, remote):
        action = self.policy(self.obs, self.policy_info)
        self.obs, self.ret, self.done, self.info = self.agent.step(action)
        vy = -robot.getRemoteState().lx
        vx = robot.getRemoteState().ly
        omega = -robot.getRemoteState().rx*2.2
        self.command_profile.x_vel_cmd = vx*1.5
        self.command_profile.y_vel_cmd = vy*1.5
        self.command_profile.yaw_vel_cmd = omega


In [ ]:
class BaseRemote:
    def __init__(self):
        pass
    def startSeq(self):
        return False
    def standUpDownSeq(self):
        return False

    def flushStates(self):
        pass

    def getEstop(self):
        return False

class UnitreeRemote(BaseRemote):
    def __init__(self, robot):
        self.robot = robot

    def startSeq(self):
        remote = self.robot.getRemoteState()
        if remote.btn.start:
            return True
        else:
            return False

    def standUpDownSeq(self):
        remote = self.robot.getRemoteState()
        if remote.btn.L2 and remote.btn.A:
            return True
        else:
            return False

    def getEstop(self):
        remote = self.robot.getRemoteState()
        if remote.btn.L2 and remote.btn.R2:
            return True
        else:
            return False
        

In [ ]:
remote = UnitreeRemote(robot)
safety_hypervisor = SafetyHypervisor(robot)

In [ ]:
checkpoint = "/home/rstaion/projects/rooholla/locomotion/Go2Py/Go2Py/assets/checkpoints/walk_these_ways/"
# checkpoint = "/home/rstaion/projects/rooholla/walk-these-ways/runs/gait-conditioned-agility/pretrain-v0/train/025417.456545"

controller = walkTheseWaysController(robot, remote, checkpoint)
fsm = FSM(robot, remote, safety_hypervisor, user_controller_callback=controller.update)
safety_hypervisor = SafetyHypervisor(robot)
fsm.control_dT = 1./56.